In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [4]:
df.sentiment.value_counts()

0    25000
1    25000
Name: sentiment, dtype: int64

In [5]:
import string

In [6]:
def sentence_to_vec(s,embedding_dict,stopwords,tokenizer):
    words = str(s).lower()
    words = tokenizer(words)
    words = [word for word in words if word not in string.punctuation]
    words = [word for word in words if word not in stopwords]
    words = [word for word in words if word.isalpha()]
    
    M = []
    for w in words:
        if w in embedding_dict:
            M.append(embedding_dict[w])
            
    if len(M)==0:
        return np.zeros(300)
    M=np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

In [7]:
def load_embeddings(file):
    f = open(file,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word]=wordEmbedding        
    
    return gloveModel

In [8]:
print('Loading embeddings')
embeddings = load_embeddings('../input/glove-embeddings/glove.6B.300d.txt')

Loading embeddings


In [9]:
from nltk.tokenize import word_tokenize

In [10]:
vectors = []
for review in df.review.values:
    vectors.append(sentence_to_vec(s= review,embedding_dict=embeddings,stopwords=[],tokenizer=word_tokenize))
    
    
vectors = np.array(vectors)
y = df.sentiment.values
print('Done')

Done


In [11]:
x_train,x_test,y_train,y_test = train_test_split(vectors,y,test_size=0.2,stratify=y)

In [12]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [13]:
logreg.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [14]:
y_pred = logreg.predict(x_test)

In [15]:
print('Accuracy of the model : {:.3f}'.format(accuracy_score(y_test,y_pred)))

Accuracy of the model : 0.825
